In [1]:
#Import libs
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from fancyimpute import IterativeImputer
import pickle

Using TensorFlow backend.


In [2]:
#Read csv
raw_df = pd.read_csv('exercise_03_train.csv')

In [3]:
#Inspect data
raw_df.head()
#There are a lot of features!  EDA will be very difficult without understanding which
#features are likely to be important. No knowledge of collection practices or data dictionaries
#provided.  Will not attempt thorough EDA since the missing aforementioned data makes doing so
#far less tractable.

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x91,x92,x93,x94,x95,x96,x97,x98,x99,y
0,83.812330,-0.122672,65.391785,6.323478,-18.511031,2.122648,0.319472,7.650422,7.713315,2.789856,...,5.482352,12.211997,asia,-3.201565,-9.837196,-21.092011,4.671140,0.588994,-11.417083,0
1,-0.445312,-0.436077,12.981759,30.716674,-34.981679,-16.712862,0.530105,8.740222,56.044533,0.075118,...,-6.083035,-5.098374,asia,5.433036,91.724841,4.305371,4.329130,0.371513,8.474528,0
2,82.927148,0.075277,-29.096012,-20.176841,10.109713,-45.994005,0.618132,7.757838,-69.749060,-1.561335,...,0.200787,-7.174907,asia,2.595005,-14.630603,-3.743052,-14.820376,-2.366675,2.613091,0
3,370.736586,-0.263338,-75.273905,-21.765844,-72.478339,-8.701108,0.332586,2.769763,-35.815423,5.230400,...,4.986738,-6.649765,asia,4.349013,-53.825281,-12.243755,-4.257462,-0.352829,-9.641206,0
4,33.775263,-0.270047,-40.486723,3.990431,-13.733621,-6.185321,-0.733206,-3.325673,-5.055036,-4.331587,...,3.793349,-0.865512,asia,4.432424,8.605707,-23.823016,-11.913495,-0.139055,-0.318227,0


In [4]:
#Seperate categorical variables
category_df = raw_df.loc[:,raw_df.dtypes == np.object]

In [5]:
#Seperate numerical variables
num_df = raw_df.loc[:,raw_df.dtypes == np.float64]

In [6]:
#summarize categorical variables
category_df.describe(include=[np.object])

,x34,x35,x41,x45,x68,x93
count,39993,39991,39994,39995,39990,39994
unique,10,8,37824,10,12,3
top,volkswagon,wed,$-1073.04,0.01%,July,asia
freq,12610,14775,4,9640,11053,35442


In [7]:
#convert categorical vars ($,%) to numeric
category_df['x41'] = category_df['x41'].str.replace('$','').astype(float)
category_df['x45'] = category_df['x45'].str.replace('%','').astype(float)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
#add converted vars to numeric df
num_df['x41'] = category_df['x41']
num_df['x45'] = category_df['x45']
category_df.drop(['x41','x45'],axis = 1, inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

In [9]:
#print counts of each categorical variable
for col in category_df:
    print(category_df[col].value_counts(dropna=False),'\n')

volkswagon    12610
Toyota        10884
bmw            7306
Honda          5176
tesla          2248
chrystler      1218
nissan          342
ford            168
mercades         29
chevrolet        12
NaN               7
Name: x34, dtype: int64 

wed          14775
thurday      13378
wednesday     5938
thur          4405
tuesday        882
friday         521
monday          66
fri             26
NaN              9
Name: x35, dtype: int64 

July       11053
Jun         9256
Aug         8173
May         4801
sept.       3514
Apr         1640
Oct          910
Mar          414
Nov          150
Feb           47
Dev           21
January       11
NaN           10
Name: x68, dtype: int64 

asia       35442
america     3124
euorpe      1428
NaN            6
Name: x93, dtype: int64 



In [10]:
#create mask to align same categories with different names
day_mask1 = category_df['x35'] == 'wednesday'
day_mask2 = category_df['x35'] == 'fri'
day_mask3 = category_df['x35']  == 'thur'

In [11]:
#align names
category_df['x35'][day_mask1] = 'wed'
category_df['x35'][day_mask2] = 'friday'
category_df['x35'][day_mask3] = 'thurday'

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:7626: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2961: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
#perform inductive, single imputation with numerical df
#NOTE: Will not attempt multiple imputation
imp = IterativeImputer()
imp_num_df = pd.DataFrame(imp.fit_transform(num_df))
imp_num_df.index = num_df.index
imp_num_df.columns = num_df.columns

In [13]:
#Save imputation fit
with open('imp', 'wb') as i:
    pickle.dump(imp, i, pickle.HIGHEST_PROTOCOL)

In [14]:
#impute missing categorical vars as most frequent in category
for col in category_df:
    max_freq = category_df[col].value_counts().index[0]
    category_df[col][pd.isna(category_df[col])] = max_freq

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:7626: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2961: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [15]:
#convert categories into nominal integers
le_dict = {}
label_df = pd.DataFrame()
for col in category_df:
    le = LabelEncoder()
    label_df[col] = le.fit_transform(category_df[col])
    le_dict[col] = le

In [16]:
#save lable enc
with open('le_dict', 'wb') as l:
    pickle.dump(le_dict, l, pickle.HIGHEST_PROTOCOL)

In [17]:
#create df of binary cols representing instance of each category across multipile columns
enc = OneHotEncoder()
onehot_df = pd.DataFrame(enc.fit_transform(label_df).toarray())
#feature_names = enc.get_feature_names()
#onehot_df.columns = feature_names

In [18]:
#save one-hot-enc
with open('enc', 'wb') as e:
    pickle.dump(enc, e, pickle.HIGHEST_PROTOCOL)

In [19]:
scalar = StandardScaler()
scaled_num_df = pd.DataFrame(scalar.fit_transform(imp_num_df))

In [20]:
#save scalar
with open('scalar', 'wb') as s:
    pickle.dump(scalar, s, pickle.HIGHEST_PROTOCOL)

In [21]:
#create df of x vars from imputed df and encoded df
x_df = pd.concat([scaled_num_df, onehot_df], axis = 1)

In [22]:
#save x vars
with open('x_df', 'wb') as x:
    pickle.dump(x_df, x, pickle.HIGHEST_PROTOCOL)

In [23]:
#sepearte dependant 'y' binary variable
y_df = raw_df['y'].astype('int')

In [24]:
#save y var
with open('y_df', 'wb') as y:
    pickle.dump(y_df, y, pickle.HIGHEST_PROTOCOL)